In [4]:
def param_count(m):
    m = m.model
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

In [19]:
from utilities.biollama import BioLlama
import time

questions = ["Which is the main calcium pump of the sarcoplasmic reticulum? Answer:"]
#answers = ["Sarcoplasmic reticulum Ca(2+)-ATPase"] # or "SERCA","serca2"

db_name = "RCT200ktrain"
retrieval_text_mode = "input_segmentation"

prompt = questions[0]
model_id = "TheBloke/Llama-2-7b-chat-GPTQ"
chunk_length = 32

time_before_setup = time.time()
BioLlama = BioLlama(model_id=model_id, chunk_length=chunk_length)


params, trainable_params = param_count(BioLlama)

# freeze layers (disable gradients)
for param in BioLlama.model.parameters(): param.requires_grad = False
for param in BioLlama.model.lm_head.parameters(): param.requires_grad = True
for param in BioLlama.model.model.layers[0:14].parameters(): param.requires_grad = False
for param in BioLlama.model.model.layers[16:].parameters(): param.requires_grad = False
BioLlama.model.model.embed_tokens.weight.requires_grad_(False)
BioLlama.model.gradient_checkpointing_enable()
params, trainable_params = param_count(BioLlama)


time_before_generation = time.time()
num_tokens, text = BioLlama.generate(prompt=prompt, max_length=33)

time_after = time.time()

print("***Generating***")
print(text)
# actual_response = text[len(prompt):]
# print(actual_response)
# print(f"Actual response length: {len(actual_response)}")
print(f"Time taken for setup: {time_before_generation - time_before_setup}")
print(f"Time taken for generation: {time_after - time_before_generation}")
print(f"Tokens per second: {num_tokens/(time_after - time_before_generation)}")
print(f"Time total: {time_after - time_before_setup}")

is in config mapping
<class 'utilities.finetuning.cti.transformers.transformers.src.transformers.models.auto.configuration_auto._LazyConfigMapping'>
that worked!
Wrapping layer 15 with retro
is in config mapping
<class 'utilities.finetuning.cti.transformers.transformers.src.transformers.models.auto.configuration_auto._LazyConfigMapping'>
that worked!
is in config mapping
<class 'utilities.finetuning.cti.transformers.transformers.src.transformers.models.auto.configuration_auto._LazyConfigMapping'>
that worked!
Total params: 393.49M, Trainable: 393.49M
Total params: 393.49M, Trainable: 131.07M
now going through decoder_layer 0
now going through decoder_layer 1
now going through decoder_layer 2
now going through decoder_layer 3
now going through decoder_layer 4
now going through decoder_layer 5
now going through decoder_layer 6
now going through decoder_layer 7
now going through decoder_layer 8
now going through decoder_layer 9
now going through decoder_layer 10
now going through decoder_

/home/service/BioLlama/utilities/finetuning/cti/transformers/transformers/src/transformers/generation/utils.py:1547: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


KeyboardInterrupt: 